In [1]:
pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.3 MB/s eta 0:00:00


In [2]:
pip install transformers -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 87.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.9 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd

In [5]:

data = pd.read_csv('/content/drive/MyDrive/aloi/balance_data_way2.csv')

data.tail()


,text,label
12333,Tôi không bao giờ từng nghĩ rằng bạn sẽ đạt đư...,4
12334,Tôi cảm thấy thực sự vui mừng với sự thành côn...,4
12335,Tôi không thể nghĩ ra một cách nào để mô tả cả...,4
12336,Tôi thật sự rất ngạc nhiên với điều đó.,4
12337,Tôi cảm thấy hạnh phúc vì bạn đã làm được điều...,4


In [6]:
data['label'].value_counts()

4    1837
0    1803
3    1772
5    1763
2    1751
1    1725
6    1687
Name: label, dtype: int64

In [7]:
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from transformers import TrainingArguments, Trainer
from transformers import RobertaForSequenceClassification, AutoTokenizer,AutoModelForSequenceClassification


In [8]:
tokenizer = AutoTokenizer.from_pretrained("wonrax/phobert-base-vietnamese-sentiment", use_fast=False)
model = RobertaForSequenceClassification.from_pretrained("wonrax/phobert-base-vietnamese-sentiment",num_labels=7, ignore_mismatched_sizes=True)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at wonrax/phobert-base-vietnamese-sentiment and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([7, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([7]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
model = model.to('cuda')

In [10]:
sample_data = ["Tui rất yêu bạn, tui thích bạn "]
tokenizer(sample_data, padding=True, truncation=True, max_length=512)

{'input_ids': [[0, 17410, 59, 413, 18964, 1301, 4, 6265, 543, 88, 2]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [11]:
X = list(data["text"])
y = list(data["label"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, random_state=42)


In [12]:
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

In [13]:
X_train_tokenized.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [14]:
print(X_train_tokenized['attention_mask'][0])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [15]:
len(X_train),len(X_val)

(10487, 1851)

In [16]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [17]:
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [18]:
train_dataset[5]

{'input_ids': tensor([    0,  3438,   599,  1500,  1658,   170,   691,  1733, 20819,    13,
             9,   575,  1500,  1582,    41,   240,    40,     6,  3224,  5130,
           351,     5,  2029,  3535,   269,   572,  5262,    90,  1596, 23845,
          1460,   351,    94,  8840,  5135,   179,    58,     4,  1236, 42247,
          1500,  1658,   366,     2,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,   

In [19]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    accuracy = accuracy_score(y_true=labels, y_pred=predictions)
    precision = precision_score(y_true=labels, y_pred=predictions, average='weighted')
    recall = recall_score(y_true=labels, y_pred=predictions, average='weighted')
    f1 = f1_score(y_true=labels, y_pred=predictions, average='weighted')

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1
    }


In [20]:
import transformers
trainer = Trainer(
        model=model,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
        args= TrainingArguments(
            output_dir='./results',
            per_device_train_batch_size=8,
            per_device_eval_batch_size=8,
            # gradient_accumulation_steps=1,
            warmup_steps=10,
            num_train_epochs=3,
            logging_steps=100,
            report_to="none",
            evaluation_strategy="steps",
            save_strategy="steps",
            save_total_limit=10,
            load_best_model_at_end=True
        ),
        data_collator= transformers.DataCollatorWithPadding(tokenizer=tokenizer,  padding = True, return_tensors="pt")
    )

In [21]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1 Score
100,1.644900,1.439862,0.490005,0.552750,0.490005,0.478923
200,1.239000,0.997217,0.666667,0.678484,0.666667,0.665331
300,1.013000,0.896056,0.693139,0.712881,0.693139,0.693814
400,0.946700,0.941239,0.686656,0.685546,0.686656,0.680909
500,0.867900,0.819602,0.716910,0.736328,0.716910,0.720669
600,0.863700,0.799360,0.725554,0.767141,0.725554,0.728633
700,0.851100,0.806076,0.728795,0.731414,0.728795,0.724784
800,0.848300,0.767755,0.739600,0.783971,0.739600,0.745514
900,0.845700,0.781997,0.733657,0.754861,0.733657,0.732184
1000,0.842200,0.697499,0.753647,0.771425,0.753647,0.757172


TrainOutput(global_step=3933, training_loss=0.5841702139471555, metrics={'train_runtime': 1679.6631, 'train_samples_per_second': 18.731, 'train_steps_per_second': 2.342, 'total_flos': 2910272528313000.0, 'train_loss': 0.5841702139471555, 'epoch': 3.0})

In [22]:
trainer.evaluate()

{'eval_loss': 0.6681790351867676,
 'eval_accuracy': 0.8055105348460292,
 'eval_precision': 0.8124580956870242,
 'eval_recall': 0.8055105348460292,
 'eval_f1_score': 0.8077668643255013,
 'eval_runtime': 13.2167,
 'eval_samples_per_second': 140.051,
 'eval_steps_per_second': 17.554,
 'epoch': 3.0}

In [ ]:
text = "người buồn thì cảnh có vui bao giờ"
# text = "go to hell"
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')
outputs = model(**inputs)
print(outputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)
predictions = predictions.cpu().detach().numpy()
predictions
#{0: "Other", 1: "Disgust", 2:"Enjoyment", 3:"Anger", 4:"Surprise", 5:"Sadness", 6:"Fear"}

SequenceClassifierOutput(loss=None, logits=tensor([[ 2.0695, -0.7758,  1.0532, -1.7038, -2.3462,  3.3238, -1.9724]],
       device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([[0.2009, 0.0117, 0.0727, 0.0046, 0.0024, 0.7042, 0.0035]],
       device='cuda:0', grad_fn=<SoftmaxBackward0>)


array([[0.20088579, 0.01167484, 0.07270624, 0.00461546, 0.00242778,
        0.7041617 , 0.00352829]], dtype=float32)

In [23]:
trainer.save_model('/content/drive/MyDrive/aloi/save_weight_balance_last')

In [ ]:
import torch

# Train your model

# Save your model to Google Drive
torch.save(model.state_dict(), '/content/drive/MyDrive/code/custome_model.pth')

In [ ]:
from transformers import RobertaForSequenceClassification, AutoTokenizer,AutoModelForSequenceClassification
import torch

model_2 = RobertaForSequenceClassification.from_pretrained("saved_weight")
tokenizer = AutoTokenizer.from_pretrained("wonrax/phobert-base-vietnamese-sentiment", use_fast=False)

##{0: "Other", 1: "Disgust", 2:"Enjoyment", 3:"Anger", 4:"Surprise", 5:"Sadness", 6:"Fear"}

In [ ]:
# text = "That was good point"
text = "thật là cảm động quá đi á"
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt')
outputs = model_2(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predictions = predictions.cpu().detach().numpy()
predictions

array([[1.2990483e-03, 9.9406826e-01, 1.4193089e-03, 1.1295922e-03,
        3.2226768e-04, 9.5252739e-04, 8.0898177e-04]], dtype=float32)

##{0: "Other", 1: "Disgust", 2:"Enjoyment", 3:"Anger", 4:"Surprise", 5:"Sadness", 6:"Fear"}

In [ ]:
def change_format_to(list_input):
    formatted_list = [[f"{value:.3f}" for value in sublist] for sublist in list_input]
    return formatted_list

text = ":) anh ghét em lắm Vy"
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt')
outputs = model_2(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predictions = predictions.cpu().detach().numpy()
output_list = change_format_to(predictions)

print(output_list)


[['0.001', '0.994', '0.001', '0.001', '0.000', '0.001', '0.001']]


In [ ]:
import plotly.express as px


# Convert strings to floats and scale up by 100
scaled_data = [float(value) * 100 for value in output_list[0]]

# Custom names for each bar
bar_names = ['Other', 'Disgust', 'Enjoyment', 'Anger', 'Surprise', 'Sadness', 'Fear']

# Create a DataFrame for the plot
import pandas as pd
df = pd.DataFrame({'Kind of Emotion': bar_names, 'Data': output_list[0], 'Scaled Value': scaled_data})

# Create the interactive bar chart using Plotly Express
fig = px.bar(df, y='Kind of Emotion', x='Scaled Value', text='Scaled Value', labels={'Scaled Value': 'Probability'})

# Add title and customize layout
# Add title and customize layout
fig.update_layout(
    title= text ,
    xaxis_title='Probability',
    yaxis_title='Kind of Emotion',
    width=800,  # Set the width of the chart
    height=400  # Set the height of the chart
)


# Show the interactive plot in Jupyter Notebook
fig.show()


In [ ]:
def find_position_of_biggest_value(lst):
    if not lst:
        return None

    max_value = lst[0]
    max_position = 0

    for i in range(1, len(lst)):
        if lst[i] > max_value:
            max_value = lst[i]
            max_position = i

    return max_position

def map_position_to_label(position):
    label_mapping = {
        0: "Other",
        1: "Disgust",
        2: "Enjoyment",
        3: "Anger",
        4: "Surprise",
        5: "Sadness",
        6: "Fear"
        # Add more mappings as needed
    }

    return label_mapping.get(position, "Unknown")

def find_biggest_value_label(lst):
    position = find_position_of_biggest_value(lst)
    if position is not None:
        label = map_position_to_label(position)
        return label
    else:
        return "List is empty"


# Example usage:
result_label = find_biggest_value_label(output_list[0])
print("Emotion of sentence is:", result_label)

Emotion of sentence is: Anger
